In [2]:
import pandas as pd
import numpy as np
import os
import datetime as dt
os.chdir(r'C:\Users\38030\OneDrive\CORNELL\2020F\AB')

In [4]:
data1=pd.read_csv('financial_sp500_test.csv')
data2=pd.read_csv('financial_sp500_test2.csv')
data3=pd.read_csv('financial_sp500_test3.csv')
data4=pd.read_csv('financial_sp500_test4.csv')

In [5]:
#data=pd.merge(data1,data2,how='outer')
#data=pd.merge(data,data3,how='outer')
#data=pd.merge(data,data4,how='outer')

In [157]:
data=pd.read_csv('financial_sp500.csv')
data.columns

Index(['gvkey', 'datadate', 'fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc',
       'datafmt', 'curcdq', 'datacqtr', 'datafqtr', 'rdq', 'atq', 'ceqq',
       'cogsq', 'dlcq', 'dlttq', 'dpq', 'dvpq', 'epspi12', 'epspiq', 'glaq',
       'ibcomq', 'ibq', 'icaptq', 'ltq', 'niitq', 'niq', 'oiadpq', 'oibdpq',
       'optdrq', 'revtq', 'saleq', 'seqq', 'wcapq', 'xintq', 'xoprq', 'xrdq',
       'costat', 'dvpsxq'],
      dtype='object')

In [158]:
data1=pd.read_csv('financial_sp500_annual.csv')
data1=data1[['gvkey','fdate','at', 'dv', 'dvt', 'ebit', 'ebitda', 'epspx', 'fincf', 'gla',
       'ivncf', 'lt', 'oancf', 'revt', 'costat']]

In [159]:
data['dvpq_3y']=data.groupby(['gvkey'])['dvpq'].transform(lambda x: x.rolling(12).mean())
data['saleq_y']=data.groupby(['gvkey'])['saleq'].transform(lambda x: x.rolling(4).mean())
data['oibdpq_y']=data.groupby(['gvkey'])['oibdpq'].transform(lambda x: x.rolling(4).mean())
data['atq_y']=data.groupby(['gvkey'])['atq'].transform(lambda x: x.rolling(4).mean())
data['revtq_y']=data.groupby(['gvkey'])['revtq'].transform(lambda x: x.rolling(4).mean())
data['niq_y']=data.groupby(['gvkey'])['niq'].transform(lambda x: x.rolling(4).mean())
data['icaptq_y']=data.groupby(['gvkey'])['icaptq'].transform(lambda x: x.rolling(4).mean())

In [45]:
price = pd.read_csv('sp500_cons_prc_new.csv')

In [46]:
price.sort_values(by=['gvkey','datadate'],inplace=True)
price['MA3']=price.groupby(['gvkey'])['trt1m'].transform(lambda x: x.rolling(3).mean())
price['MA6']=price.groupby(['gvkey'])['trt1m'].transform(lambda x: x.rolling(6).mean())
price['MA12']=price.groupby(['gvkey'])['trt1m'].transform(lambda x: x.rolling(12).mean())
price['EMA3']=price.groupby(['gvkey'])['trt1m'].transform(lambda x: x.ewm(3).mean())
price['EMA6']=price.groupby(['gvkey'])['trt1m'].transform(lambda x: x.ewm(6).mean())
price['EMA12']=price.groupby(['gvkey'])['trt1m'].transform(lambda x: x.ewm(12).mean())

In [47]:
price['std12']=price.groupby(['gvkey'])['trt1m'].transform(lambda x: x.rolling(12).std())
price['std6']=price.groupby(['gvkey'])['trt1m'].transform(lambda x: x.rolling(6).std())

In [48]:
len(price)

87694

In [49]:
price.columns

Index(['adj_close', 'adj_ret', 'cheqvm', 'conm', 'cshtrm', 'datadate', 'gind',
       'gsubind', 'gvkey', 'iid', 'prccm', 'prchm', 'prclm', 'tic', 'trfm',
       'trt1m', 'cshom', 'MA3', 'MA6', 'MA12', 'EMA3', 'EMA6', 'EMA12',
       'std12', 'std6'],
      dtype='object')

In [102]:
price['mkt_cap']=price['prccm']*price['cshom']

## Valuation

In [167]:
try:
    data.drop(columns=['indfmt', 'consol', 'popsrc','costat','iid'],inplace=True)
except:
    1
data.drop(columns=['datadate'],inplace=True)
data.drop_duplicates(['gvkey','rdq'],keep='last',inplace=True)

In [161]:
rdq=data[['rdq','gvkey']].drop_duplicates()
month_date=price['datadate'].drop_duplicates()

In [162]:
rdq_month=pd.DataFrame()
for d in month_date:
    temp=rdq[rdq.rdq<=d]
    temp=temp.groupby(['gvkey'])['rdq'].max().reset_index()
    temp['datadate']=d
    rdq_month=rdq_month.append(temp)

In [169]:
price_new=pd.merge(price,rdq_month,how='left')
price_new.columns

Index(['adj_close', 'adj_ret', 'cheqvm', 'conm', 'cshtrm', 'datadate', 'gind',
       'gsubind', 'gvkey', 'iid', 'prccm', 'prchm', 'prclm', 'tic', 'trfm',
       'trt1m', 'cshom', 'MA3', 'MA6', 'MA12', 'EMA3', 'EMA6', 'EMA12',
       'std12', 'std6', 'mkt_cap', 'rdq'],
      dtype='object')

In [172]:
price_new=pd.merge(price_new,data)

In [93]:
price_new.to_csv('price+financial.csv',index=False)

## dividend+annual

In [174]:
dividend = pd.read_csv('dividend_monthly.csv')
dividend=dividend[['gvkey','datadate','dvpspm','dvpsxm']]
dividend.drop_duplicates(['gvkey','datadate'],inplace=True)

In [175]:
#price_new=pd.read_csv('price+financial.csv')
price_new=pd.merge(price_new,dividend,how='left')
price_new['dvpspm'].fillna(0,inplace=True)
price_new['dvpsxm'].fillna(0,inplace=True)
#data.to_csv('price+financial.csv',index=False)

In [177]:
data1.columns

Index(['gvkey', 'fdate', 'at', 'dv', 'dvt', 'ebit', 'ebitda', 'epspx', 'fincf',
       'gla', 'ivncf', 'lt', 'oancf', 'revt', 'costat'],
      dtype='object')

In [179]:
fdate=data1[['fdate','gvkey']].drop_duplicates()
month_date=price_new['datadate'].drop_duplicates()
fdate_month=pd.DataFrame()
for d in month_date:
    temp=fdate[fdate.fdate<=d]
    temp=temp.groupby(['gvkey'])['fdate'].max().reset_index()
    temp['datadate']=d
    fdate_month=fdate_month.append(temp)

In [181]:
price_new1=pd.merge(price_new,fdate_month,how='left')

In [182]:
data1.drop_duplicates(['gvkey','fdate'],keep='last',inplace=True)
price_new1=pd.merge(price_new1,data1)

In [237]:
price_new1.to_csv('price+financial.csv',index=False)

## Features

In [216]:
# Compute ROE
price_new1['ROE'] = price_new1['niq']/(price_new1['cshom']*price_new1['prccm'])*1e6
# Compute ROA --> 'niq'/'atq'
price_new1['ROA'] = price_new1['niq']/price_new1['atq']

# Compute Leverage --> (DLTT + DLC)/SEQ --> no price_new1
# Need to download "DLTTQ","DLCQ","SEQQ" in WRDS -> "Fundamental Quarterly"
price_new1['leverage'] = (price_new1['dlttq']+price_new1['dlcq'])/price_new1['seqq']

# Compute EPS --> already have column 'epspi12'

# Compute Growth --> Net income growth rate? 
# Question: ni is quarter price_new1, so ni_growth will have many 0 values
# Even if you change it to "OIBDPQ" (Operating Income), it still has the same problem
price_new1['NI_Growth'] = price_new1.groupby(['gvkey'])['niq_y'].transform(lambda x: (x-x.shift(1))/x.shift(1))

# Compute Net Profit Margin ---> Net Income/Revenue
price_new1['NetPM'] = price_new1['niq']/price_new1['revtq']

# Compute price/cap
price_new1['PriceCap'] = price_new1['prccm']/price_new1['mkt_cap']

# Compute Earnings vol (operating risk)
# Earnings = (Net income - preferred dividends) = ('niq' - 'dvpq') (need 'dvpq' price_new1!)
price_new1['earnings_q'] = (price_new1['niq']-price_new1['dvpq'])*1e6
price_new1['earningsVol'] = price_new1.groupby(['gvkey'])['earnings_q'].transform(lambda x: x.rolling(4).std())

# Compute ROI --> 'niq'/ 'icaptq'(invested capital, no price_new1 yet)
price_new1['ROI'] = price_new1['prccm']/price_new1['icaptq']/1e6

# Compute DPS --> Dividends per Share / Ex Date / Monthly --> 'dvpsxm'

# Compute Asset Turnover --> Revenue/Average Assets
# Need 'atq' price_new1 from WRDS
price_new1['Asset Turnover'] = price_new1['revtq']/price_new1['atq_y']

# Compute Size
price_new1['Size'] = np.log(price_new1['mkt_cap'])

In [234]:
price_new1['PE']=price_new1['prccm']/price_new1['epspi12']
price_new1['PB']=price_new1['adj_close']/(price_new1['atq']-price_new1['ltq'])*price_new1['cshom']/1e6
price_new1['PS']=price_new1['prccm']/price_new1['saleq_y']*price_new1['cshom']/1e6
price_new1['dividend_yield']=price_new1.groupby(['gvkey'])['dvpsxm'].transform(lambda x: x.rolling(12).sum())/price_new1['adj_close']
#price_new1['PCF']=price_new1['prccm']/(price_new1['oiadpq']-)

In [223]:
price_new1['EV']=price_new1['mkt_cap']+price_new1['ltq']*1e6
price_new1['ev_ebitda']=price_new1['EV']/price_new1['ebitda']/1e6
price_new1['net_cash_flow']=(price_new1['fincf']+price_new1['ivncf']+price_new1['oancf'])*1e6
price_new1['NCFP']=price_new1['net_cash_flow']/price_new1['mkt_cap']

In [228]:
price_new1['industry_id']=price_new1['gind']//10000
price_new1.to_csv('price+financial.csv',index=False)#11 industries

In [211]:
print(list(price_new1.columns)[50:])

['niitq', 'niq', 'oiadpq', 'oibdpq', 'optdrq', 'revtq', 'saleq', 'seqq', 'wcapq', 'xintq', 'xoprq', 'xrdq', 'costat', 'dvpsxq', 'dvpq_3y', 'saleq_y', 'oibdpq_y', 'atq_y', 'revtq_y', 'niq_y', 'icaptq_y', 'dvpspm', 'dvpsxm', 'fdate', 'at', 'dv', 'dvt', 'ebit', 'ebitda', 'epspx', 'fincf', 'gla', 'ivncf', 'lt', 'oancf', 'revt', 'ROE', 'ROA', 'leverage', 'NI_Growth', 'NetPM', 'PriceCap', 'earnings_q', 'earningsVol', 'ROI', 'Asset Turnover', 'Size', 'PE', 'PB', 'PS', 'dividend_yield', 'EV', 'eb_ebitda', 'net_cash_flow', 'NCFP', 'industry_id']


## Features Selection

In [235]:
features=price_new1[['gvkey','tic','conm','datadate','industry_id','trt1m','adj_ret','adj_close',
                     'cshtrm','prccm','prchm','prclm','MA3','MA6','MA12','EMA3','EMA6','EMA12','std12','std6',
                     'Size','PriceCap','ROE','ROA','ROA','earnings_q', 'earningsVol','Asset Turnover',
                     'PE', 'PB', 'PS', 'dividend_yield','ev_ebitda','NCFP']]
sp500_part=pd.read_csv('sp500_cons_part_prc.csv')
sp500_part=sp500_part[['gvkey','datadate']].drop_duplicates()
features=pd.merge(features,sp500_part)
features.to_csv('sp500_features.csv',index=False)

In [236]:
features['dividend_yield']

0        0.005627
1        0.008201
2        0.009346
3        0.009916
4        0.007407
           ...   
59892    0.009869
59893    0.009739
59894    0.008699
59895    0.008410
59896    0.008582
Name: dividend_yield, Length: 59897, dtype: float64